#1.安装OPENAI






In [1]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 3.4 MB/s eta 0:00:00


#2.配置API KEY作为环境变量
需要使用OPENAI的 API，新注册的用户有5 刀的赠送值，限期1个月使用。超过一个月需要通过信用卡充值。

In [2]:
import os
os.environ["OPENAI_API_KEY"] = "[you key]"


#3.准备好要嵌入的数据集
这里需要prompt 和 completion的方式，其实就是一问一答的方式。将生成的训练集输出到外部文件，准备训练


In [3]:
import json
import openai

training_data = [{
    "prompt": "账单在哪里？ ->",
    "completion": " 在桌子上.\n"
},{
    "prompt":"How do I upgrade my account ->",
    "completion": " Visit you user settings in the left-hand side menu, then click 'upgrade account' button at the top.\n"
}]

file_name = "training_data.jsonl"

with open(file_name, "w") as output_file:
 for entry in training_data:
  json.dump(entry, output_file)
  output_file.write("\n")


#4.检查训练数据
这里通过OPEN AI提供的工具，对要训练的数据进行检查。
包括：给你的建议，微调的文件名字，需要多长时间等信息。

In [4]:
!openai tools fine_tunes.prepare_data -f training_data.jsonl


Analyzing...

- Your file contains 2 prompt-completion pairs. In general, we recommend having at least a few hundred examples. We've found that performance tends to linearly increase for every doubling of the number of examples
- All prompts end with suffix ` ->`
- All completions end with suffix `.\n`

No remediations found.

You can use your file for fine-tuning:
> openai api fine_tunes.create -t "training_data.jsonl"

After you’ve fine-tuned a model, remember that your prompt has to end with the indicator string ` ->` for the model to start generating completions, rather than continuing with the prompt. Make sure to include `stop=[".\n"]` so that the generated texts ends at the expected place.
Once your model starts training, it'll approximately take 2.47 minutes to train a `curie` model, and less for `ada` and `babbage`. Queue will approximately take half an hour per job ahead of you.


#5.上传训练数据
这里需要上传训练集的数据文件到OPENAI，然后有OPENAI 进行微调。
微调之后会生成一个新的训练文件，这个文件名需要记好，后面进行问答的时候需要使用。

In [ ]:

upload_response = openai.File.create(
  file=open(file_name, "rb"),
  purpose='fine-tune'
)
file_id = upload_response.id
upload_response

<File file id=file-EaMYyi30eFpSIsptu8ZziQOT at 0x7f544508f470> JSON: {
  "object": "file",
  "id": "file-EaMYyi30eFpSIsptu8ZziQOT",
  "purpose": "fine-tune",
  "filename": "file",
  "bytes": 295,
  "created_at": 1688863790,
  "status": "uploaded",
  "status_details": null
}

#6.进行微调
默认事curie 模型微调，当然也可以设置需要的模型
openai.FineTune.create(training_file=file_id, model="davinci")


"Davinci" 是 OpenAI 提供的一种预训练的大型语言模型。OpenAI 提供了多种不同的预训练模型，包括 "Davinci"、"Curie"、"Babbage" 和 "Ada"，这些模型在大小和能力上有所不同，"Davinci" 是其中最大、最强大的模型。


In [ ]:
fine_tune_response = openai.FineTune.create(training_file="file-EaMYyi30eFpSIsptu8ZziQOT",model="davinci")
fine_tune_response

#7.检查微调过程

超参数（hyperparameters）是在开始学习过程之前设置的参数，而不是通过训练得到的参数。换句话说，这些参数是我们在开始训练之前就需要决定的，它们会影响学习过程（即模型训练的方式）以及模型的性能（如准确性）。

"n_epochs"：表示模型训练的轮数。一轮（epoch）意味着模型已经看过了所有的训练数据一次。例如，如果n_epochs=10，那么模型将会遍历整个训练数据集10次。

"batch_size"：表示每次模型训练时使用的样本数量。例如，如果batch_size=32，那么模型将会在每次更新参数时使用32个样本。

"prompt_loss_weight"：表示在计算总损失时，提示损失的权重。这个参数可以用来平衡不同类型的损失。

"learning_rate_multiplier"：表示学习率的乘数。学习率是一个决定模型学习速度的参数，乘数可以用来调整学习率。

In [9]:
#retrieve_response = openai.FineTune.retrieve(id=fine_tune_response.id)
retrieve_response = openai.FineTune.retrieve(id='ft-WnnCrcb1k5tBzbmOyBb1pshA')

retrieve_response

<FineTune fine-tune id=ft-WnnCrcb1k5tBzbmOyBb1pshA at 0x7c78e5bfc4a0> JSON: {
  "object": "fine-tune",
  "id": "ft-WnnCrcb1k5tBzbmOyBb1pshA",
  "hyperparams": {
    "n_epochs": 4,
    "batch_size": 1,
    "prompt_loss_weight": 0.01,
    "learning_rate_multiplier": 0.1
  },
  "organization_id": "org-4P7htKo6DejPTQxfu3rExc7D",
  "model": "davinci",
  "training_files": [
    {
      "object": "file",
      "id": "file-EaMYyi30eFpSIsptu8ZziQOT",
      "purpose": "fine-tune",
      "filename": "file",
      "bytes": 295,
      "created_at": 1688863790,
      "status": "processed",
      "status_details": null
    }
  ],
  "validation_files": [],
  "result_files": [
    {
      "object": "file",
      "id": "file-ur43CZKsO6htJtnokGHFYox6",
      "purpose": "fine-tune-results",
      "filename": "compiled_results.csv",
      "bytes": 527,
      "created_at": 1688873228,
      "status": "processed",
      "status_details": null
    }
  ],
  "created_at": 1688863815,
  "updated_at": 1688873229,

In [6]:
import openai


openai.FineTune.list()

<OpenAIObject list at 0x785882cee610> JSON: {
  "object": "list",
  "data": [
    {
      "object": "fine-tune",
      "id": "ft-2gEiNZ2SrsyzxsyYAKHOKDr0",
      "hyperparams": {
        "n_epochs": 4,
        "batch_size": 1,
        "prompt_loss_weight": 0.01,
        "learning_rate_multiplier": 0.1
      },
      "organization_id": "org-4P7htKo6DejPTQxfu3rExc7D",
      "model": "curie",
      "training_files": [
        {
          "object": "file",
          "id": "file-2w4lcjtupdkpPiebpLMD0HB4",
          "purpose": "fine-tune",
          "filename": "file",
          "bytes": 272,
          "created_at": 1688819288,
          "status": "processed",
          "status_details": null
        }
      ],
      "validation_files": [],
      "result_files": [
        {
          "object": "file",
          "id": "file-Ejg6dAootRPlLNs1xjVsfg7n",
          "purpose": "fine-tune-results",
          "filename": "compiled_results.csv",
          "bytes": 491,
          "created_at": 16888261

#8.测试新模型

In [9]:
#retrieve_response = openai.FineTune.retrieve(fine_tune_response.id)
retrieve_response = openai.FineTune.retrieve('ft-WnnCrcb1k5tBzbmOyBb1pshA')

fine_tuned_model = retrieve_response.fine_tuned_model
fine_tuned_model

'davinci:ft-personal-2023-07-09-03-27-07'

In [10]:
#将训练完的模型保存到fine_tuned_model中

new_prompt = "账单在哪里？ ->"

answer = openai.Completion.create(
  model=fine_tuned_model,
  prompt=new_prompt,
  max_tokens=10,
  temperature=0
)
#answer
answer['choices'][0]['text']

' 在桌子上\n\n'